In [ ]:
import argparse, os, sys, time, shutil
from     pathlib import Path
from collections import Counter
from    datetime import datetime, timedelta

parser = argparse.ArgumentParser()
parser.add_argument("--device",         type=str,   default='cuda',  help="Specify 'cpu' or 'cuda'")
parser.add_argument("--cuda_vd",        type=str,   default='None',  help="CUDA_VISIBLE_DEVICES")
parser.add_argument("--seed",           type=int,   default=999,     help="Seed value")
parser.add_argument("--epochs",         type=int,   default=400,     help="The number of epochs")
parser.add_argument("--esep",           type=int,   default=200,     help="The number of early stop epoch")
parser.add_argument("--ep_eval",        type=int,   default=50,      help="The number of epoch to start evaluation")
parser.add_argument("--bsize",          type=int,   default=10,      help="Batch size")
parser.add_argument("--WRSns",          type=int,   default=3000,    help="Specify number of batch to active WeightedRandomSampler")
parser.add_argument("--num_workers",    type=int,   default=0,       help="The number of workers for DataLoader")
parser.add_argument("--lr",             type=float, default=-1,      help="Learning rate, -1 means 0.1, -2 means 0.01")
parser.add_argument("--lr_lmbda",       type=float, default=-1,      help="The value of lr_lmbda for MultiplicativeLR, -1 means 0.1")
parser.add_argument("--lr_step_wait",   type=int,   default=50,      help="learning rate step wait epochs")
parser.add_argument("--opt",            type=str,   default='sgd',   help="optimizer name")
parser.add_argument("--cp",             type=float, default='inf',   help="To clips gradient norm")
parser.add_argument("--momentum",       type=float, default=0,       help="momentum value")
parser.add_argument("--weight_decay",   type=float, default=0,       help="The value of weight_decay for optimizer")
parser.add_argument("--scheduler",      type=str,   default='None',  help="scheduler name, None or MultiplicativeLR")
parser.add_argument("--dmode_train",    type=str,   default='train', help="dmode for  train Dataset")
parser.add_argument("--dmode___dev",    type=str,   default='eval',  help="dmode for    dev Dataset")
parser.add_argument("--dmode__eval",    type=str,   default='eval',  help="dmode for   eval Dataset")
# parser.add_argument("--path_data",      type=str,   required=True,   help="Specify path of ASVspoof2019 directory")
# parser.add_argument("--task",           type=str,   default='PA',    help="Specify task of ASVspoof2019, LA or PA")
# parser.add_argument("--conifg_section", type=str,   required=True,   help="Specify config section for different features")
parser.add_argument("--info",           type=str,                    help="additional message")
args = parser.parse_args(args=[])

args.cuda_vd = '0'
args.dmode_train = 'fixed'
args.dmode___dev = 'fixed'
args.dmode__eval = 'fixed'
args.path_data = '../ASVspoof2019'
args.task = 'PA'
args.conifg_section = 'QTAC'

In [ ]:
import   configparser
config = configparser.ConfigParser()
config.read('config.ini')

dim_f          = config.getint(args.conifg_section, 'dim_f')
dim_t          = config.getint(args.conifg_section, 'dim_t')
feature_folder = config.get(   args.conifg_section, 'feature_folder')
file_extension = config.get(   args.conifg_section, 'file_extension')

from dataset import get_list_dict_task_online as get_list_dict
list_path_train, dict_cm_train, \
list_path___dev, dict_cm___dev, asv_data__dev, \
list_path__eval, dict_cm__eval, asv_data_eval= \
get_list_dict(Path(args.path_data), args.task, feature_folder, file_extension)
from dataset import Dataset_online as Data_set
dataset_dev = Data_set(list_path___dev, dict_cm___dev, config, args.conifg_section, args.dmode___dev)
dataset_eva = Data_set(list_path__eval, dict_cm__eval, config, args.conifg_section, args.dmode__eval)

In [ ]:
import torch, random
import torch.backends.cudnn as cudnn
import torch.nn as nn
import numpy    as np
from torch.utils.data import DataLoader
device = torch.device(args.device)
random.seed(      int(args.seed))
np.random.seed(   int(args.seed))
torch.manual_seed(int(args.seed))
cudnn.deterministic = True

class__counts___dev = Counter(dict_cm___dev.values())
counts__class___dev = [class__counts___dev[False], class__counts___dev[True]]
class_weights___dev = [1 - (x / class__counts___dev.total()) for x in counts__class___dev]
Dloader_dev = DataLoader(dataset_dev, batch_size=args.bsize, shuffle=False, num_workers=args.num_workers, collate_fn=None)
nl_criterion___dev = nn.CrossEntropyLoss(torch.tensor(class_weights___dev, dtype=torch.float32)).to(device)

nl_criterion = nn.CrossEntropyLoss().to(device)
Dloader_eva = DataLoader(dataset_eva, batch_size=args.bsize, shuffle=False, num_workers=args.num_workers, collate_fn=None)

In [ ]:
from model import T45_LCNN as Model
model = Model(data_shape=[dim_f, dim_t], LDO_p1=0.75, LDO_p2=0.00).to(device)

In [ ]:
# from torchinfo import summary
# summary(model, input_size=(args.bsize, 1, dim_f, dim_t))

In [ ]:
ep = -1
save_folder_name = 'exp__QTAC'
save_folder_path = Path(os.getcwd()).home() / save_folder_name
MODL_PATH = save_folder_path / 'modl' / f'modl__ep_{ep:03d}.pt'
model.load_state_dict(torch.load(MODL_PATH))

In [ ]:
model.eval()

In [ ]:
from train_eval_infer import spf_det_eval
from em.em_2019 import get_eer, get_tDCF

time_start = time.time()
loss___dev_avg, name___dev, scrs___dev, cmky___dev = spf_det_eval(model, device, Dloader_dev, nl_criterion___dev)
time_dev = time.time() - time_start

bona__cm___dev = scrs___dev[cmky___dev == 'bonafide']
spoof_cm___dev = scrs___dev[cmky___dev == 'spoof']
min_tDCF___dev = get_tDCF(asv_data__dev, bona__cm___dev, spoof_cm___dev)
eer___cm___dev = get_eer(bona__cm___dev, spoof_cm___dev)[0] * 100

if min_tDCF___dev == 0: str_first_tdcf = f'dev tDCF EER:{            " 0":<6s}'
else:                   str_first_tdcf = f'dev tDCF EER:{min_tDCF___dev:>7.4f}'

if eer___cm___dev == 0: str_secnd      = f'{str_first_tdcf}{            " 0":<6s}'
else:                   str_secnd      = f'{str_first_tdcf}{eer___cm___dev:>7.4f}'

print(f'{str_secnd}  time-dev:{str(timedelta(seconds=int(time_dev))):>8s}')

In [ ]:
LAPADF = 'PA'
ASV_2021_root = Path(os.getcwd()).parent / 'ASVspoof2021'
ASV_2021__dir = f'ASVspoof2021_{LAPADF}_eval'
ASV_2021_filelist_path = ASV_2021_root / ASV_2021__dir / f'ASVspoof2021.{LAPADF}.cm.eval.trl.txt'
ASV_2021_filelist = list(np.genfromtxt(ASV_2021_filelist_path, dtype=str))
data_path_predict = ASV_2021_root / ASV_2021__dir / feature_folder
list_path_predict = [data_path_predict / f'{name}.{file_extension}' for name in ASV_2021_filelist]
print(list_path_predict[:3])

In [ ]:
from train_eval_infer import spf_det_infer

dataset_2021_eval = Data_set(list_path_predict, 'infer', config, args.conifg_section, args.dmode__eval)
Dloader_2021_eval = DataLoader(dataset_2021_eval, batch_size=args.bsize, shuffle=False, num_workers=args.num_workers, collate_fn=None)

time_start = time.time()
name_predict, scrs_predict = spf_det_infer(model, device, Dloader_2021_eval)
time_2021_eval = time.time() - time_start
print(f' time-2021-{LAPADF}-eval:{str(timedelta(seconds=int(time_2021_eval))):>8s}')

In [ ]:
print(len(ASV_2021_filelist))
print(len(name_predict))
print(len(scrs_predict))
assert len(name_predict) == len(scrs_predict)

print(max(scrs_predict))
print(min(scrs_predict))
np.save(f'ASVspoof2021-{LAPADF}-scrs-eval-{args.conifg_section}-name', name_predict)
np.save(f'ASVspoof2021-{LAPADF}-scrs-eval-{args.conifg_section}', scrs_predict)

with open(f'ASVspoof2021-{LAPADF}-scrs-eval-{args.conifg_section}.txt', 'w') as f:
    for i in range(len(name_predict)):
        _ = f.write(f'{name_predict[i]} {scrs_predict[i]}\n')